In [ ]:
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time

In [1]:
# 0228 day 1
# 착한가게 데이터 selenium 으로 크롤링하기
# 원하는 데이터 컬럼명 설정해서 추출하기

driver_path = "C:\chromedriver_win32/chromedriver.exe"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito')

browser = webdriver.Chrome(driver_path, chrome_options = chrome_options)
browser.get('https://선한영향력가게.com/store/view?idx=1079')


# 403Error 방지: url접근변수 설정
current_url = browser.current_url
headers = {'User-Agent':'Mozilla/5.0'}
req = Request(current_url, headers = headers)

# http source 추출을 위한 변수설정
html = urlopen(req)
source = html.read().decode("utf-8")
code = html.getcode()

# http 출력로직
if code == 200 :
    print("url 정상 추출 코드 : 200\n")
else :
    print("HTTP-Error")

# html을 dom화 하여서 객체화
soup = BeautifulSoup (source)


# 문서의 제목확인
print ("문서의 제목: ", soup.title.text, type(soup))

# 식당정보가 담겨있는 태그수
stores = soup.select("div.store-desc")
print ("식당 정보 담겨있는 태그 수 : ", len(stores), type(stores))

stores_info=[]

for store in stores:
    store_name = store.select("em.underline")
    store_address = store.select("div.store-desc>dl address")
    store_tel = store.select("div.store-desc>dl a")
    store_div = store.select("div.store-desc>dl address")
    store_runtime = store.select("div.store-desc > dl > div:nth-child(2) > dd > div")
    store_for = store.select("div.store-desc > dl > div:nth-child(3) > dd")
    
    if len(store_name) > 0 :
        stores_info.append({
            "식당 이름" : store_name[0].text,
            "식당 주소" : store_address[0].text,
            "식당 번호" : store_tel[0].text,
            "영업 시간" : store_runtime[0].text.replace("\u200b",""),
            "제공 대상" : store_for[0].text.replace("\r\n","").strip()
        })
print(stores_info, type(stores_info))

for i in range (1, 2):
    url_1="https://선한영향력가게.com/store/view?idx="
    browser.get(url_1+str(i))
    
    try:
        WebDriverWait (browser, 1).until(EC.alert_is_present())
        alert= browser.switch_to.alert
        alert.accept()
        
    except:
        # http source 추출을 위한 변수설정
        html = urlopen(req)
        source = html.read().decode("utf-8")
        code = html.getcode()

        soup = BeautifulSoup (source)

        html = urlopen(browser.current_url)
        soup = BeautifulSoup (html.read().decode("utf-8"))
        stores = soup.select("div.store-desc")


        for store in stores:
            store_name = store.select("em.underline")
            store_address = store.select("div.store-desc dl address")
            store_tel = store.select("div.store-desc>dl a")
            store_div = store.select("div.store-desc>dl address")
            store_runtime = store.select("div.store-desc > dl > div:nth-child(2) > dd > div")
            store_for = store.select("div.store-desc > dl > div:nth-child(3) > dd")

            stores_info.append({
                    "식당 이름" : store_name[0].text,
                    "식당 주소" : store_address[0].text,
                    "식당 번호" : store_tel[0].text if len(store_tel) > 0 else "",
                    "영업 시간" : store_runtime[0].text.replace("\u200b","") if len(store_runtime) > 0 else "",
                    "제공 대상" : store_for[0].text.replace("\r\n","").strip()if len(store_runtime) > 0 else ""
                })
# else browser.switchTo().alert().accept() :  
        
print(stores_info, type(stores_info))

# str1= "오전오후시분"
# stores_info1("영업 시간").replace(str1,"")

stores=pd.DataFrame(stores_info)
stores.to_csv("C:/mini_project/stores.csv", header=False, index=False)
stores

seoul_stores=stores[stores["식당 주소"].str.contains("서울")]
seoul_stores


C:\Users\BIT\AppData\Local\Temp\ipykernel_1060\2819396815.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(driver_path, chrome_options = chrome_options)
C:\Users\BIT\AppData\Local\Temp\ipykernel_1060\2819396815.py:15: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(driver_path, chrome_options = chrome_options)


url 정상 추출 코드 : 200

문서의 제목:  선한영향력가게::대연생고깃간 <class 'bs4.BeautifulSoup'>
식당 정보 담겨있는 태그 수 :  1 <class 'bs4.element.ResultSet'>
[{'식당 이름': '대연생고깃간', '식당 주소': '서울 성북구 돌곶이로22나길 29 1층 대연생고깃간', '식당 번호': '010-8475-6567', '영업 시간': '오전9시30분~ 저녁9시30분', '제공 대상': '아이 본인만                                                                                                                                                                                                                                                    결식아동'}] <class 'list'>
[{'식당 이름': '대연생고깃간', '식당 주소': '서울 성북구 돌곶이로22나길 29 1층 대연생고깃간', '식당 번호': '010-8475-6567', '영업 시간': '오전9시30분~ 저녁9시30분', '제공 대상': '아이 본인만                                                                                                                                                                                                                                                    결식아동'}] <class 'list'>


,식당 이름,식당 주소,식당 번호,영업 시간,제공 대상
0,대연생고깃간,서울 성북구 돌곶이로22나길 29 1층 대연생고깃간,010-8475-6567,오전9시30분~ 저녁9시30분,아이 본인만 ...
